In [1]:
!pip install -r requirements.txt

  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached decorator-5.0.7-py3-none-any.whl (8.8 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
Processing c:\users\aarbs\appdata\local\pip\cache\wheels\b8\ae\ae\c63bae74a3b3e18b2fa35e179387e6cdeb03a1c479ed45c351\en_core_web_md-2.3.1-py3-none-any.whl
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Using cached ipykernel-5.5.3-py3-none-any.whl (120 kB)
  Using cached ipython-7.22.0-py3-none-any.whl (785 kB)
  Using cached ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
  Using cached jedi-0.18.0-py2.py3-none-any.whl (1.4 MB)
  Using cached Jinja2-2.11.3-py2.py3-none-any.whl (125 kB)
  Using cached jupyter_client-6.1.12-py3-none-any.whl (112 kB)
  Using cached jupyter_core-4.7.1-py3-none-any.whl (82 kB)
  Using cached jupyterlab_widgets-1.0.0-py3-none-any.whl (243 kB)
  Using ca

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\aarbs\\anaconda3\\Lib\\site-packages\\~ornado\\speedups.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached prometheus_client-0.10.1-py2.py3-none-any.whl (55 kB)
  Using cached prompt_toolkit-3.0.18-py3-none-any.whl (367 kB)
  Using cached protobuf-3.15.8-py2.py3-none-any.whl (173 kB)
  Using cached pydeck-0.6.2-py2.py3-none-any.whl (4.2 MB)
  Using cached Pygments-2.8.1-py3-none-any.whl (983 kB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached pyzmq-22.0.3-cp38-cp38-win_amd64.whl (1.2 MB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
  Using cached soupsieve-2.2.1-py3-none-any.whl (33 kB)
  Using cached spacy-2.3.2-cp38-cp38-win_amd64.whl (9.5 MB)
  Using cached srsly-1.0.5-cp38-cp38-win_amd64.whl (178 kB)
  Using cached streamlit-0.77.0-py2.py3-none-any.whl (7.5 MB)
  Using cached terminado-0.9.4-py3-none-any.whl (14 kB)
  Using cached thinc-7.4.1-cp38-cp38-win_amd64.whl (2.1 MB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached tornado-6.1-cp38-cp38-win_amd64.wh

In [2]:
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
import random

In [3]:
def load_data(train_data, limit=0, split=0.8):
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{"FAKE": not bool(y), "REAL": bool(y)} for y in labels]
    split = int(len(train_data) * split)

    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


In [4]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "FAKE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

In [6]:
nlp = spacy.load('en_core_web_md')
df = pd.read_csv('COVID Fake News Data.csv')
df.replace(to_replace='[\n\r\t]', value=' ', regex=True, inplace=True)

In [7]:
textcat=nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [8]:
textcat.add_label("REAL")
textcat.add_label("FAKE")

1

In [9]:
df['tuples'] = df.apply(lambda row: (row['headlines'], row['outcome']), axis=1)
train = df['tuples'].tolist()

In [10]:
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(train, split=0.9)
train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))

In [11]:
n_iter = 20
# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()

    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

      # Calling the evaluate() function and printing the scores
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

Training the model...
LOSS 	  P  	  R  	  F  
3.504	0.987	0.925	0.955
1.534	0.969	0.925	0.946
0.996	0.961	0.929	0.945
0.746	0.965	0.929	0.947
0.366	0.957	0.937	0.947
0.254	0.962	0.941	0.951
0.156	0.966	0.937	0.951
0.211	0.961	0.933	0.947
0.232	0.962	0.941	0.951
0.249	0.962	0.941	0.951
0.192	0.970	0.941	0.955
0.095	0.958	0.946	0.952
0.128	0.966	0.946	0.956
0.109	0.970	0.950	0.960
0.133	0.950	0.954	0.952
0.075	0.950	0.954	0.952
0.037	0.966	0.946	0.956
0.158	0.966	0.950	0.958
0.025	0.966	0.954	0.960
0.083	0.974	0.950	0.962


In [12]:
with nlp.use_params(optimizer.averages):
            nlp.to_disk('model')